In [2]:
// SETUP
use std::fs::File;
use std::collections::{HashMap, HashSet};
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};
use std::time::Instant;

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

fn test_file(problem_id: u32, test_id: u32) -> String {
    format!("{}/{:02}t{}.txt", DATA_ROOT, problem_id, test_id)
}

In [65]:
// DAY01
const TOTAL: i64 = 2020;

fn problem01() {    
    let nums: Vec<i64> = read_integers(&data_file(1)).unwrap();    
    let mut reg = HashMap::new();

    for (i, n) in nums.iter().enumerate() {
        let rest = TOTAL - n;
        if reg.contains_key(&rest) {
            println!("{} * {} = {}", n, rest, n * rest);
        }
        reg.insert(n, i);
    }
    
    for (i, n1) in nums.iter().enumerate() {
        for j in (i + 1)..nums.len() {
            let n2 = nums[j];
            let rest = TOTAL - n1 - n2;
            match reg.get(&rest) {
                Some(k) if k > &j => {
                    println!("{} * {} * {} = {}", n1, n2, rest, n1 * n2 * rest)
                },
                _ => (),
            }
        }
    }
}

problem01();

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [50]:
// DAY02
#[derive(Debug, PartialEq)]
struct PasswordDesc {
    cnt1: usize,
    cnt2: usize,
    ch: char,
    pwd: String,
}

impl FromStr for PasswordDesc {
    type Err = std::num::ParseIntError;
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let parts: Vec<&str> = s.split(' ').collect();
        let counts: Vec<usize> = parts[0]
            .split('-')
            .map(|p| p.parse().unwrap())
            .collect();
        Ok(PasswordDesc { 
            cnt1: counts[0], 
            cnt2: counts[1], 
            ch: parts[1].as_bytes()[0] as char, 
            pwd: parts[2].to_owned()})
    }
}

fn is_valid1(p: &PasswordDesc) -> bool {
    let cnt = p.pwd.matches(p.ch).count();
    p.cnt1 <= cnt && cnt <= p.cnt2
}

fn is_valid2(p: &PasswordDesc) -> bool {
    let pwd = p.pwd.as_bytes();
    let ch = p.ch as u8;
    (pwd[p.cnt1 - 1] == ch) ^ (pwd[p.cnt2 - 1] == ch)
}

fn problem02() {
    let passwords: Vec<PasswordDesc> = 
        read_lines(&data_file(2)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();

    println!("Number of valid passwords: {}, {}", 
        passwords.iter().filter(|x| is_valid1(x)).count(), 
        passwords.iter().filter(|x| is_valid2(x)).count());
}

problem02();

Number of valid passwords: 548, 502


In [72]:
// DAY03
const TREE: u8 = '#' as u8;

fn count_trees(rows: &Vec<String>, dx: usize, dy: usize) -> usize {
    let h = rows.len();
    if h == 0 {
        return 0;
    }
    let w = rows[0].len();
    let mut x: usize = 0;
    let mut res: usize = 0;
    for y in (0..h).step_by(dy) {
        res += (rows[y].as_bytes()[x % w] == TREE) as usize;
        x += dx;
    }
    res
}

fn problem03() {
    let rows = read_lines(&data_file(3)).unwrap();
    let res1 = count_trees(&rows, 3, 1);
    println!("Num trees 1: {}", res1);
    
    let res2: usize = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
        .iter()
        .map(|(dx, dy)| count_trees(&rows, *dx, *dy))
        .product();
    println!("Num trees 2: {}", res2);
}

problem03();

Num trees 1: 242
Num trees 2: 2265549792


In [93]:
// DAY04
#[macro_use] extern crate lazy_static;
extern crate regex;

use regex::Regex;
use std::fs;
use std::collections::HashSet;

type Passport = HashMap<String, String>;

fn read_passport(line: &str) -> Passport {
    line.split_whitespace()
        .filter_map(|s| {
            let mut parts = s.split(':');
            Some((parts.next()?.to_owned(), parts.next()?.to_owned())) 
        })
        .collect()
}

const REQUIRED_FIELDS: [&str; 7] = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"];

fn is_valid1(passport: &Passport) -> bool {
    REQUIRED_FIELDS.iter().all(|f| passport.contains_key(*f))    
}

fn is_valid2(passport: &Passport) -> bool {
    lazy_static! {
        static ref PID_REGEX: Regex = Regex::new(r"^(\d{9})$").unwrap();
        static ref HCL_REGEX: Regex = Regex::new(r"^#[0-9a-f]{6}$").unwrap();
        static ref HGT_REGEX: Regex = Regex::new(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$").unwrap();
        static ref EYE_COLORS: HashSet<String> = ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
            .iter()
            .copied()
            .map(|x| x.to_owned())
            .collect();
    }
    
    fn in_range<T: Ord + FromStr>(int_str: &str, minv: T, maxv: T) -> bool {
        match int_str.parse::<T>() {
            Ok(n) => n >= minv && n <= maxv,
            _ => false
        }
    }
    
    fn is_valid_height(height_str: &str) -> bool {
        match HGT_REGEX.captures(height_str) {
            Some(c) => 
                match (c.name("in"), c.name("cm")) {
                    (Some(x), _) => in_range(x.as_str(), 59, 76),
                    (_, Some(x)) => in_range(x.as_str(), 150, 193),
                    _ => false,
                },
            _ => false
        }
    }
    
    is_valid1(passport) &&
    in_range(&passport["byr"], 1920, 2002) &&
    in_range(&passport["iyr"], 2010, 2020) &&
    in_range(&passport["eyr"], 2020, 2030) &&
    EYE_COLORS.contains(passport["ecl"].as_str()) &&
    PID_REGEX.is_match(&passport["pid"]) &&
    HCL_REGEX.is_match(&passport["hcl"]) &&
    is_valid_height(&passport["hgt"])
}

fn problem04() {
    let passports: Vec<Passport> = fs::read_to_string(&data_file(4))
        .unwrap()
        .split("\n\n")
        .map(|s| read_passport(s))
        .collect();

    println!("Valid passports 1: {}", passports.iter().filter(|x| is_valid1(x)).count());
    println!("Valid passports 2: {}", passports.iter().filter(|x| is_valid2(x)).count());
}

problem04();

Valid passports 1: 264
Valid passports 2: 224


In [50]:
// DAY05
use std::collections::HashSet;
use std::iter::FromIterator;

fn get_id(seat: &str) -> u32 {
    seat.chars().fold(0, |res, c| res * 2 + "BR".contains(c) as u32)
}

fn problem05() {
    let lines = read_lines(&data_file(5)).unwrap();
    let ids: Vec<u32> = lines
        .iter()
        .map(|x| get_id(x))
        .collect();
    
    let max_id: u32 = *ids.iter().max().unwrap();
    let id_set = ids.iter().cloned().collect::<HashSet<u32>>();
    
    let mut seat: u32 = 0;
    for i in 0..=(1 << lines[0].len()) {
        let id = i as u32;
        if !id_set.contains(&id) && 
            id_set.contains(&(id + 1)) && 
            id_set.contains(&(id - 1)) {
            seat = id;
            break;
        }
    }
    println!("Max ID: {}, Seat: {}", max_id, seat);
}

problem05();

Max ID: 801, Seat: 597


In [44]:
// DAY06
use std::fs;
use std::collections::HashMap;

fn problem06() {
    let text = fs::read_to_string(&data_file(6)).unwrap();
    let groups: Vec<Vec<&str>> = text
            .split("\n\n")
            .map(|s| s.split_whitespace().collect())
            .collect();
    
    let (mut res1, mut res2) = (0, 0);
    for group in &groups {
        let mut counts: HashMap<char, usize> = HashMap::new();
        for answers in group {
            for ans in answers.chars() {
                *counts.entry(ans).or_insert(0) += 1;
            }
        }
        res1 += counts.len();
        res2 += counts.iter().filter(|&(_, v)| *v == group.len()).count()
    }
    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem06();

Answer 1: 6310, Answer 2: 3193


In [63]:
// DAY07
type Mapping = HashMap<String, HashMap<String, u32>>;
type BackMapping = HashMap<String, Vec<String>> ;

fn parse_mapping(line: &str) -> (String, HashMap<String, u32>) {
    let parts: Vec<&str> = line.split_whitespace().collect();
    let bag_color = parts[0..=1].join(" ");
    let mut res: HashMap<String, u32> = HashMap::new();
    for i in (4..parts.len()).step_by(4) {
        if parts[i] != "no" {
            let child_bag_color = parts[i + 1..i + 3].join(" ");
            res.insert(child_bag_color, parts[i].parse::<u32>().unwrap());
        }
    }
    (bag_color, res)
}

fn build_back_mapping(mapping: &Mapping) -> BackMapping {
    let mut res: HashMap<_, _> = HashMap::new();
    for (key, children) in mapping {
        for (child, _) in children {
            res.entry(child.clone()).or_insert(Vec::new()).push(key.clone());
        }
    }
    res
}

fn count_reachable(root: &str, back_mapping: &BackMapping) -> u32 {
    fn rec(node: &str, back_mapping: &BackMapping, visited: &mut HashSet<String>) -> u32 {
        if visited.contains(node) {
            return 0;
        }
        let mut res = 1;
        if back_mapping.contains_key(node) {
            for child in &back_mapping[node] {
                res += rec(&child, back_mapping, visited);
            }
        }
        visited.insert(node.to_owned());
        res
    }
    let mut visited: HashSet<String> = HashSet::new();
    rec(root, back_mapping, &mut visited)
}

fn count_contains(root: &str, mapping: &Mapping) -> u32 {
    let mut res: u32 = 1;
    for (child, count) in &mapping[root] {
        res += count * count_contains(&child, mapping);
    }
    res
}

fn problem07() {
    let mapping: Mapping = read_lines(&data_file(7))
        .unwrap()
        .iter()
        .map(|line| parse_mapping(line))
        .into_iter()
        .collect();
    let back_mapping = build_back_mapping(&mapping);
    const ROOT: &str = "shiny gold";
    println!("Reachable: {}", count_reachable(ROOT, &back_mapping) - 1);
    println!("Contains: {}", count_contains(ROOT, &mapping) - 1);
}

problem07();

Reachable: 370
Contains: 29547


In [43]:
// DAY08

#[derive(Debug, PartialEq, Copy, Clone)]
enum Op {
    Acc(i64),
    Nop(i64),
    Jmp(i64),
}

impl FromStr for Op {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let mut it = s.split(' ');
        let opstr: &str = it.next().ok_or(())?;
        let val: i64 = it.next().ok_or(())?.parse::<i64>().map_err(|_|())?;
        match opstr {
            "acc"  => Ok(Op::Acc(val)),
            "nop"  => Ok(Op::Nop(val)),
            "jmp"  => Ok(Op::Jmp(val)),
            _      => Err(()),
        }
    }
}

fn eval_ops(ops: &Vec<Op>) -> (bool, i64) {
    let n = ops.len() as i64;
    let mut visited: Vec<bool> = Vec::new();
    visited.resize(ops.len(), false);
    let mut ip: i64 = 0;
    let mut acc: i64 = 0;
    while ip >= 0 && ip < n {
        if visited[ip as usize] {
            return (false, acc)
        }
        visited[ip as usize] = true;
        match ops[ip as usize] {
            Op::Jmp(val) => ip += val - 1,
            Op::Acc(val) => acc += val,
            _ => (),
        }
        ip += 1;
    }
    (true, acc)
}

fn try_mutate_program(ops: &Vec<Op>) -> Option<i64> {
    let mut ops_copy = ops.clone();
    for (i, op) in ops.iter().enumerate() {
        let prevOp: Op = *op;
        let newOp = match op {
            Op::Jmp(val) => Op::Nop(*val),
            Op::Nop(val) => Op::Jmp(*val),
            _ => prevOp
        };
        if prevOp != newOp {
            ops_copy[i] = newOp;
            let (terminated, acc) = eval_ops(&ops_copy);
            if terminated {
                return Some(acc);
            }
            ops_copy[i] = prevOp;
        }
    }
    None
}

fn problem08() {
    let ops: Vec<Op> = 
        read_lines(&data_file(8)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();
    let (_, acc) = eval_ops(&ops);
    println!("Answer 1: {}", acc);
    println!("Answer 2: {}", try_mutate_program(&ops).unwrap());
}

problem08();

Answer 1: 1782
Answer 2: 797


In [111]:
// DAY09
use std::collections::BTreeMap;

fn get_first_non_sum(nums: &Vec<i64>, k: usize) -> Option<i64> {
    // use btree data structure to maintain a sorted window of size k
    let mut window = BTreeMap::new();
    let n = nums.len();
    if n < k {
        return None;
    }
    for i in 0..k {
        *window.entry(nums[i]).or_insert(1) += 1;
    }
    for i in k..n {
        let s = nums[i];
        let (mut it_l, mut it_r) = (window.iter(), window.iter().rev());
        let (mut l, mut r) = (it_l.next()?, it_r.next()?);
        while l.0 != r.0 {
            let cur_sum = l.0 + r.0;
            if cur_sum < s {
                l = it_l.next()?;
            } else if cur_sum > s {
                r = it_r.next()?;
            } else {
                break;
            }
        }
        if l.0 == r.0 && *l.1 == 1 {
            return Some(s)
        }
        
        let to_remove = nums[i - k];
        window.entry(to_remove).and_modify(|e| { *e -= 1 });
        if window[&to_remove] == 0 {
            window.remove(&to_remove);
        }
        *window.entry(s).or_insert(1) += 1;
    }
    None
}

fn find_sum_range(nums: &Vec<i64>, s: i64) -> Option<(usize, usize)>{
    let n = nums.len();
    if n == 0 {
        return None
    }
    let (mut l, mut r) = (0, 0);
    let mut cur_sum = nums[0];
    loop {
        if cur_sum < s {
            r += 1;
            cur_sum += nums[r];
        } else if cur_sum > s {
            cur_sum -= nums[l];
            l += 1;
        } else {
            break;
        }
        if r == n || l == n {
            break;
        }
    }
    if l != r {
        Some((l, r))
    } else {
        None
    }
}

fn find_sum_range_agg(nums: &Vec<i64>, s: i64) -> Option<i64>{
    let range = find_sum_range(&nums, s)?;
    let slice = &nums[range.0..=range.1];
    let max = slice.iter().max()?;
    let min = slice.iter().min()?;    
    Some(min + max)
}

fn problem09() {
    const WINDOW_SIZE: usize = 25;
    let nums: Vec<i64> = read_integers(&data_file(9)).unwrap();
    let non_sum = get_first_non_sum(&nums, WINDOW_SIZE).unwrap();
    println!("First non-sum: {}", non_sum);
    println!("Sum range: {}", find_sum_range_agg(&nums, non_sum).unwrap());
}

problem09();

First non-sum: 85848519
Sum range: 13414198


In [131]:
// DAY10

fn problem10() {
    let mut nums: Vec<i64> = read_integers(&data_file(10)).unwrap();
    nums.sort();
    nums.insert(0, 0);
    nums.push(nums.iter().max().unwrap() + 3);
    
    let n = nums.len();
    let mut diffs: Vec<i64> = Vec::new();
    for i in 1..n {
        diffs.push(nums[i] - nums[i - 1]);
    }
    
    let diffs1: i64 = diffs.iter().map(|x| (*x == 1) as i64).sum();
    let diffs3: i64 = diffs.iter().map(|x| (*x == 3) as i64).sum();
    
    println!("Sum 1/3 diffs: {}", diffs1 * diffs3);
    
    fn count_arrangements(nums: &Vec<i64>) -> i64 {
        fn rec(nums: &Vec<i64>, idx: usize, counts: &mut Vec<i64>) -> i64 {
            let n = nums.len();
            if idx == n - 1 {
                return 1;
            }
            if counts[idx] >= 0 {
                return counts[idx];
            }
            let mut res = 0;
            let mut i = idx;
            while i < n - 1 {
                i += 1;
                let diff = nums[i] - nums[idx];
                if diff <= 3 {
                    res += rec(nums, i, counts);
                } else {
                    break;
                }
            }
            counts[idx] = res;
            res
        }
        let mut counts: Vec<i64> = Vec::new();
        counts.resize(nums.len(), -1);
        rec(nums, 0, &mut counts)
    }
    
    println!("Number of arrangements: {}", count_arrangements(&nums));
}

problem10();

Sum 1/3 diffs: 2414
Number of arrangements: 21156911906816


In [38]:
// DAY11
use std::fmt;

#[derive(Debug, PartialEq, Copy, Clone)]
enum Seat {
    Floor,
    Vacant,
    Busy,
}

type SeatPlan = Vec<Vec<Seat>>;

impl FromStr for Seat {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        match s {
            "."  => Ok(Seat::Floor),
            "L"  => Ok(Seat::Vacant),
            "#"  => Ok(Seat::Busy),
            _      => Err(()),
        }
    }
}

const OFFS: [(i32, i32); 8] = [
    (-1, -1), (0, -1), (1, -1), 
    (-1, 0), (1, 0),
    (-1, 1), (0, 1), (1, 1)];

fn get_seat(seats: &SeatPlan, (x, y): (i32, i32)) -> Option<Seat> {
    let (h, w) = (seats.len() as i32, seats[0].len() as i32);
    if x >= 0 && x < w && y >= 0 && y < h {
        Some(seats[y as usize][x as usize])
    } else {
        None
    }
}

fn get_neighbor_counts(seats: &SeatPlan) -> Vec<Vec<u32>> {
    let (h, w) = (seats.len() as i32, seats[0].len() as i32);
    let mut res = vec![vec![0; w as usize]; h as usize];
    for y in 0..h {
        for x in 0..w {
            res[y as usize][x as usize] = OFFS
                .iter()
                .filter_map(|(dx, dy)| get_seat(&seats, (x + dx, y + dy)))
                .map(|s| (s == Seat::Busy) as u32)
                .sum();
        }
    }
    res
}

fn get_visible_neighbor_counts(seats: &SeatPlan) -> Vec<Vec<u32>> {
    let (h, w) = (seats.len() as usize, seats[0].len() as usize);
    let mut res = vec![vec![0; w]; h];
    let mut can_see = vec![vec![false; w]; h];
    
    for (dx, dy) in OFFS.iter() {
        for j in 0..h {
            for i in 0..w {
                let x: usize = if *dx <= 0 {i} else {w - i - 1};
                let y: usize = if *dy <= 0 {j} else {h - j - 1};
                let pos = (x as i32 + dx, y as i32 + dy);
                let visible = match get_seat(&seats, pos) {
                    Some(Seat::Floor) => can_see[pos.1 as usize][pos.0 as usize],
                    Some(Seat::Busy) => true,
                    Some(Seat::Vacant) => false,
                    _ => false
                };
                can_see[y][x] = visible;
                res[y][x] += visible as u32;
            }
        }
    }
    res
}

fn step(seats: &SeatPlan, 
        new_seats:&mut SeatPlan, 
        neighbors_fn: fn(&SeatPlan) -> Vec<Vec<u32>>, 
        max_occupied: u32) -> (u32, u32) {
    let (h, w) = (seats.len() as usize, seats[0].len() as usize);
    let neighbor_counts = neighbors_fn(seats);
    let mut num_changed = 0;
    let mut total_occupied = 0;
    for y in 0..h {
        for x in 0..w {
            let n = neighbor_counts[y][x];
            let status = match seats[y][x] {
                Seat::Vacant if n == 0 => Seat::Busy,
                Seat::Busy if n >= max_occupied => Seat::Vacant,
                val => val
            };
            new_seats[y][x] = status;
            num_changed += (status != seats[y][x]) as u32;
            total_occupied += (status == Seat::Busy) as u32;
        }
    }
    (num_changed, total_occupied)
}

fn iter_seats(seats: &SeatPlan, 
    neighbors_fn: fn(&SeatPlan) -> Vec<Vec<u32>>, 
    max_occupied: u32) -> u32 {
    let mut s1 = seats.clone();
    let mut s2 = seats.clone();
    let mut flip = true;
    loop {
        let (r1, r2) = if flip {(&s1, &mut s2)} else {(&s2, &mut s1)};
        let (changed, occupied) = step(r1, r2, neighbors_fn, max_occupied);
        if changed == 0 {
            return occupied;
        }
        flip = !flip;
    }
}

fn problem11() {
    let seats: SeatPlan = read_lines(&data_file(11))
        .unwrap()
        .iter()
        .map(|line| line.chars().map(|c| c.to_string().parse().unwrap()).collect())
        .collect();
    println!("Answer 1: {}", iter_seats(&seats, get_neighbor_counts, 4));
    println!("Answer 2: {}", iter_seats(&seats, get_visible_neighbor_counts, 5));
    
}

problem11();

Answer 1: 2211
Answer 2: 1995


In [40]:
// DAY12
#[derive(Debug, PartialEq, Copy, Clone)]
enum Action {
    North(i32),
    South(i32),
    East(i32),
    West(i32),
    Forward(i32),
    Left(i32),
    Right(i32),
}

impl FromStr for Action {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let c = s.as_bytes()[0] as char;
        let val = s[1..].parse::<i32>().map_err(|_|())?;      
        match c {
            'N' => Ok(Action::North(val)),
            'S' => Ok(Action::South(val)),
            'E' => Ok(Action::East(val)),
            'W' => Ok(Action::West(val)),
            'F' => Ok(Action::Forward(val)),
            'L' => Ok(Action::Left(val)),
            'R' => Ok(Action::Right(val)),
            _      => Err(()),
        }
    }
}

fn manhattan((x, y): (i32, i32)) -> i32 {
    x.abs() + y.abs()
}

fn move_dir((x, y): (i32, i32), dir: i32, val: i32) -> (i32, i32) {
    let ang = (dir as f32).to_radians();
    let ca = ang.cos().round() as i32;
    let sa = ang.sin().round() as i32;
    (x + val * ca, y + val * sa)
}

fn step1((x, y, dir): (i32, i32, i32), cmd: &Action) -> (i32, i32, i32) {
    match cmd {
        Action::North(val) => (x, y - val, dir),
        Action::South(val) => (x, y + val, dir),
        Action::West(val) => (x - val, y, dir),
        Action::East(val) => (x + val, y, dir),
        Action::Right(val) => (x, y, (dir + val) % 360),
        Action::Left(val) => (x, y, (dir - val + 360) % 360),
        Action::Forward(val) => {
            let (x1, y1) = move_dir((x, y), dir, *val);
            (x1, y1, dir)
        }
    }
}

fn rotate((x, y): (i32, i32), angle: i32) -> (i32, i32) {
    let ar = (angle as f32).to_radians();
    let (ca, sa) = (ar.cos(), ar.sin());
    let (xf, yf)  = (x as f32, y as f32);
    let x1 = (ca * xf - sa * yf).round() as i32;
    let y1 = (sa * xf + ca * yf).round() as i32;
    (x1, y1)
}

fn step2((x, y, wx, wy): (i32, i32, i32, i32), cmd: &Action) -> (i32, i32, i32, i32) {
    match cmd {
        Action::North(val) => (x, y, wx, wy - val),
        Action::South(val) => (x, y, wx, wy + val),
        Action::West(val) => (x, y, wx - val, wy),
        Action::East(val) => (x, y, wx + val, wy),
        Action::Right(val) => {
            let (wx1, wy1) = rotate((wx, wy), *val);
            (x, y, wx1, wy1)
        }
        Action::Left(val) => {
            let (wx1, wy1) = rotate((wx, wy), -*val);
            (x, y, wx1, wy1)
        }
        Action::Forward(val) => (x + wx * val, y + wy * val, wx, wy)
    }
}

fn problem12() {
    let commands: Vec<Action> = read_lines(&data_file(12))
        .unwrap()
        .iter()
        .map(|line| line.parse().unwrap())
        .collect();
    
    let (x1, y1, angle1) = commands.iter().fold((0, 0, 0), |acc, cmd| step1(acc, cmd));
    println!("Answer 1: {}", manhattan((x1, y1)));
    
    let (x2, y2, wx2, wy2) = commands.iter().fold((0, 0, 10, -1), |acc, cmd| step2(acc, cmd));
    println!("Answer 2: {}", manhattan((x2, y2)));
}

problem12();

Answer 1: 508
Answer 2: 30761


In [3]:
// DAY13
fn part1(periods: &Vec<(u64, u64)>, ts: u64) -> u64 {
    let mut min_diff = None;
    let mut res = 0;
    for (p, i) in periods {
        let d = p - ts % p;
        if min_diff == None || d < min_diff.unwrap() {
            min_diff = Some(d);
            res = d * p;
        }
    }
    res
}

fn part2(periods: &Vec<(u64, u64)>) -> u64 {
    let (mut p0, i0) = periods[0];
    // Assuming here that the largest number has an offset 
    // which is present as another number and they are also coprime.
    // This is something I noticed for my data, but should be safe for any input.
    p0 *= i0; 
    let mut k = 1;
    loop {
        let ts = k * p0 - i0;
        let mut found = true;
        for j in 1..periods.len() {
            let (p, i) = periods[j];
            if (ts + i) % p != 0 {
                found = false;
                break;
            }
        }
        if found {
            return ts;
        }
        k += 1;
        if k % 1000000000 == 0 {
            println!("Processed timestamp: {}", ts);
        }
    }
}

fn problem13() {
    let now = Instant::now();

    let lines = read_lines(&data_file(13)).unwrap();
    let ts: u64 = lines[0].parse::<u64>().unwrap();
    let mut periods: Vec<(u64, u64)> = lines[1].split(",")
        .map(|p| p.parse::<u64>())
        .enumerate()
        .filter_map(|(i, p)| match p {
            Ok(val) => Some((val, i as u64)),
            _ => None
        })
        .collect();
    periods.sort_by(|a, b| b.0.partial_cmp(&a.0).unwrap());
    
    println!("Answer 1: {:?}", part1(&periods, ts));
    println!("Answer 2: {:?}", part2(&periods));

    let elapsed = now.elapsed();
    println!("Elapsed: {:.2?}", elapsed);
}

problem13();

Answer 1: 153
Processed timestamp: 12960999987026
Processed timestamp: 25921999987026
Processed timestamp: 38882999987026
Processed timestamp: 51843999987026
Processed timestamp: 64804999987026
Processed timestamp: 77765999987026
Processed timestamp: 90726999987026
Processed timestamp: 103687999987026
Processed timestamp: 116648999987026
Processed timestamp: 129609999987026
Processed timestamp: 142570999987026
Processed timestamp: 155531999987026
Processed timestamp: 168492999987026
Processed timestamp: 181453999987026
Processed timestamp: 194414999987026
Processed timestamp: 207375999987026
Processed timestamp: 220336999987026
Processed timestamp: 233297999987026
Processed timestamp: 246258999987026
Processed timestamp: 259219999987026
Processed timestamp: 272180999987026
Processed timestamp: 285141999987026
Processed timestamp: 298102999987026
Processed timestamp: 311063999987026
Processed timestamp: 324024999987026
Processed timestamp: 336985999987026
Processed timestamp: 3499469999

In [90]:
// DAY14
#[macro_use] extern crate lazy_static;
extern crate regex;

use regex::Regex;

#[derive(Debug, PartialEq)]
struct MemOp {
    mask: String,
    ops: Vec<(i64, i64)>,
}

fn parse_ops(data: &str) -> Vec<MemOp> {
    lazy_static! {
        static ref MEM_REGEX: Regex = 
            Regex::new(r"^mem\[(?P<addr>\d+)\] = (?P<value>\d+)$").unwrap();
    }
    
    fn get_match_val(cap: &regex::Captures, group_name: &str) -> Option<i64> {
        match cap.name(group_name)?.as_str().parse::<i64>() {
            Ok(val) => Some(val),
            _ => None
        }
    }
    
    fn parse_op(lines: &Vec<&str>) -> Option<MemOp> {
        if lines.len() <= 1 {
            return None;
        }
        Some(MemOp {
            mask: lines[0].to_owned(), 
            ops: lines[1..]
                .iter()
                .filter_map(|line| {
                    let c = MEM_REGEX.captures(line)?;
                    Some((get_match_val(&c, "addr")?, get_match_val(&c, "value")?))
                })
                .collect()
        })
    }
    
    data.split("mask = ")
        .filter_map(|chunk| parse_op(&chunk.split("\n").collect()))
        .collect()
}

fn apply_mask1(val: &i64, mask: &str) -> i64 {
    let mut res = *val;
    let mut bit = 1;
    for i in (0..mask.len()).rev() {
        match mask.as_bytes()[i] as char {
            '0' => res = res & !bit,
            '1' => res = res | bit,
            _ => (),
        };            
        bit = bit << 1;
    }
    res
}

fn eval1(mem_ops: &Vec<MemOp>) -> HashMap<i64, i64> {
    let mut mem = HashMap::new();
    for mem_op in mem_ops {
        for (addr, val) in &mem_op.ops {
            mem.insert(*addr, apply_mask1(val, &mem_op.mask));
        }
    }
    mem
}

fn apply_mask2(val: &i64, mask: &str) -> Vec<i64> {
    fn rec(val: i64, mask: &str, pos: usize, res: &mut Vec<i64>) {
        if pos == mask.len() {
            res.push(val);
            return;
        }
        let bit = 1 << (mask.len() - pos - 1);
        let c = mask.as_bytes()[pos] as char;
        if c == '0' {
            rec(val, mask, pos + 1, res);
        }
        if c == 'X' {
            rec(val & !bit, mask, pos + 1, res);
        }
        if c == '1' || c == 'X' {
            rec(val | bit, mask, pos + 1, res);
        }
    }
    let mut res: Vec<i64> = Vec::new();
    rec(*val, mask, 0, &mut res);
    res
}

fn eval2(mem_ops: &Vec<MemOp>) -> HashMap<i64, i64> {
    let mut mem = HashMap::new();
    for mem_op in mem_ops {
        for (addr, val) in &mem_op.ops {
            for masked_addr in apply_mask2(addr, &mem_op.mask) {
                mem.insert(masked_addr, *val);
            }
        }
    }
    mem
}

fn problem14() {
    let data = std::fs::read_to_string(&data_file(14)).unwrap();
    let ops = parse_ops(&data);
    println!("Answer 1: {:?}", eval1(&ops).values().sum::<i64>());
    println!("Answer 2: {:?}", eval2(&ops).values().sum::<i64>());
}

problem14();

Answer 1: 8471403462063
Answer 2: 2667858637669


In [108]:
// DAY15
fn eval(seed: &Vec<u64>, stop_k: u64) -> u64 {
    let mut positions = HashMap::new();
    let mut next_num = 0;
    let mut k: usize = 1;
    loop {
        let num = if k <= seed.len() {seed[k - 1]} else {next_num};
        if k as u64 == stop_k {
            return num;
        }
        match positions.get(&num) {
            Some(&x) => next_num = (k as u64) - x,
            _ => next_num = 0,
        };
        positions.insert(num, k as u64);
        k += 1;
    }
}

fn problem15() {
    let seed = std::fs::read_to_string(&data_file(15)).unwrap()
        .split(",")
        .map(|x| x.parse::<u64>().unwrap())
        .collect(); 
    println!("Answer 1: {}", eval(&seed, 2020));
    println!("Answer 2: {}", eval(&seed, 30000000));
}

problem15();

Answer 1: 412
Answer 2: 243
